### Etapas
1. Criação de features
2. Tratamento de outliers
3. Normalização dos dados
4. Pipeline de pré-processamento
5. Separação em treino/teste
6. Balanceamento dos dados
7. Treinamento dos modelos

In [2]:
# @title Importação das bibliotecas utilizadas no programa

!pip install catboost

import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

# Carregamento do dataset
import requests
from pathlib import Path

# Criação de features
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Normalização dos dados
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Treinamento do modelo
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier,  AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import  AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00


In [3]:
# @title Carregamento do dataset

# Versão Google Collab
github_link = "https://github.com/mfigueireddo/ciencia-de-dados/blob/ba579573c5b8a9246ca04f7da29bc2c74c8b362c/datasets/pre-pipeline_wildfires.parquet"
url = github_link.replace("/blob/", "/raw/")

local_file_path = Path("/content/raw_wildfires.parquet")

# Faz uma requisição HTTP GET ao GitHub
with requests.get(url, stream=True) as request:

    request.raise_for_status() # Confere se houve êxito

    with open(local_file_path , "wb") as file:
        for chunk in request.iter_content(chunk_size=1024*1024):
            if chunk:
                file.write(chunk)

wildfires = pd.read_parquet(local_file_path,  engine="pyarrow") # Leitura realizada com a engine pyarrow


# Versão VSCode
# file_path = "../datasets/pre-pipeline_wildfires.parquet"
#
# wildfires = pd.read_parquet(
#     file_path,
#     engine="pyarrow",
#     use_nullable_dtypes=False
# )

In [4]:
# @title Variáveis globais

data_column_name = 'data'
id_column_name = 'fire_id'
latitude_column_name = 'latitude'
longitude_column_name = 'longitude'
precipitation_column_name = 'precipitacao'
max_temperature_column_name = 'temperatura_max'
precipitation_sum_window_column_name = 'soma_precipitacao_90_dias'
max_temperature_mean_column_name = 'media_temp_max_90_dias'
season_column_name = 'estacao_ano_id'
region_column_name = 'regiao_incendio'
target_column_name = 'houve_incendio'
indice_queima ='indice_queima'

### 1. Criação de features
<u>**Features criadas**</u>
- Estação do ano
- Região do incêndio
- Soma da precipitação nos últimos X dias
- Média de temperatura máxima nos últimos X dias

**Observação**: as features estação do ano baseada e região do incêndio não estão sendo utilizadas para treinar o modelo porque elas pioraram consideravelmente o resultado<br>


In [5]:
# @title Classe FeatureCreation

window_days = 90

class FeaturesCreation(BaseEstimator, TransformerMixin):

    def __init__(self):
        # Colunas utilizadas
        self.m_date_column = data_column_name
        self.m_group_column = id_column_name
        self.m_latitude_column = latitude_column_name
        self.m_longitude_column = longitude_column_name
        self.m_precipitation_column = precipitation_column_name
        self.m_max_temperature_column = max_temperature_column_name

        # Parâmetros personalizados para criação das features
        self.m_precipitation_window_days = window_days
        self.m_max_temperature_window_days = window_days

        # Parâmetros do DBSCAN
        self.mm_max_radiuskm = 1.0
        self.m_min_samples = 5

        # Objetos aprendidos no fit
        self.m_dbscan = None
        self.m_nearest_neighbors_core = None
        self.m_core_labels = None
        self.m_max_radius = None

    # Conversão necessária para o DBSCAN
    @staticmethod
    def convert_to_radians(latitude_or_longitude):
        return np.radians(latitude_or_longitude.astype(float))

    @staticmethod
    def convert_month_to_season(month):
        if month in (12, 1, 2): return 0  # Inverno
        if month in (3, 4, 5): return 1  # Primavera
        if month in (6, 7, 8): return 2  # Verão
        return 3  # 9, 10, 11 -> Outono

    def fit(self, dataframe, target=None):
        dataframe = dataframe.copy()

        # Desativa o DBSCAN caso não haja latitude e longitude
        missing_cols = [column for column in [self.m_latitude_column, self.m_longitude_column] if column not in dataframe.columns]
        if missing_cols:
            self.m_dbscan = None
            self.m_nearest_neighbors_core = None
            self.m_core_labels = None
            self.m_max_radius = None
            return self

        latitude_or_longitude = dataframe[[self.m_latitude_column, self.m_longitude_column]].to_numpy()
        latitude_or_longitude_radians = self.convert_to_radians(latitude_or_longitude)

        earth_radius = 6371.0
        self.m_max_radius = self.mm_max_radiuskm / earth_radius

        dbscan = DBSCAN(eps=self.m_max_radius, min_samples=self.m_min_samples, metric='haversine')
        dbscan.fit(latitude_or_longitude_radians)
        self.m_dbscan = dbscan

        # Treina um NearestNeighbors apenas nos pontos-core para atribuição de novos pontos à clusters existentes em transform()
        core_mask = np.zeros_like(dbscan.labels_, dtype=bool)
        if hasattr(dbscan, 'core_sample_indices_') and len(dbscan.core_sample_indices_) > 0:
            core_mask[dbscan.core_sample_indices_] = True
            core_points = latitude_or_longitude_radians[core_mask]
            core_labels = dbscan.labels_[core_mask]

            if len(core_points) > 0:
                nearest_neighbors = NearestNeighbors(n_neighbors=1, metric='haversine')
                nearest_neighbors.fit(core_points)
                self.m_nearest_neighbors_core = nearest_neighbors
                self.m_core_labels = core_labels
            else:
                self.m_nearest_neighbors_core = None
                self.m_core_labels = None
        else:
            self.m_nearest_neighbors_core = None
            self.m_core_labels = None

        return self

    # Rotula novos pontos
    def assign_dbscanlabels(self, dataframe):

        # Se não tivemos lat/lon ou DBSCAN treinado, devolve NaN
        if self.m_nearest_neighbors_core is None or self.m_core_labels is None or self.m_max_radius is None:
            return pd.Series([-1] * len(dataframe), index=dataframe.index, dtype='int64')

        latitude_or_longitude = dataframe[[self.m_latitude_column, self.m_longitude_column]].to_numpy()
        latitude_or_longitude_radians = self.convert_to_radians(latitude_or_longitude)

        # Atribui rótulo do core mais próximo, desde que dentro do raio
        distances, indices = self.m_nearest_neighbors_core.kneighbors(latitude_or_longitude_radians, n_neighbors=1, return_distance=True)
        distances = distances.reshape(-1)
        indices = indices.reshape(-1)

        labels = np.full(len(dataframe), -1, dtype='int64')
        within = distances <= self.m_max_radius
        labels[within] = self.m_core_labels[indices[within]]

        return pd.Series(labels, index=dataframe.index, dtype='int64')

    # Adiciona médias móveis e soma pro grupo de incêndio
    def add_temporal_rollings(self, dataframe):

        # Ordena por grupo e tempo para garantir rolling correto
        if self.m_group_column in dataframe.columns and self.m_date_column in dataframe.columns:
            dataframe = dataframe.sort_values([self.m_group_column, self.m_date_column])
        else:
            # Se faltar algo, só ordena por data (se existir)
            if self.m_date_column in dataframe.columns:
                dataframe = dataframe.sort_values(self.m_date_column)

        # Rolling de precipitação (soma dos últimos 14 dias)
        if self.m_precipitation_column in dataframe.columns:
            dataframe[precipitation_sum_window_column_name] = (
                dataframe.groupby(self.m_group_column, dropna=False)[self.m_precipitation_column]
                  .rolling(self.m_precipitation_window_days, min_periods=1)
                  .sum()
                  .reset_index(level=0, drop=True)
            )
        else:
            dataframe[precipitation_sum_window_column_name] = np.nan

        # Rolling de temperatura máxima (média dos últimos 7 dias)
        if self.m_max_temperature_column in dataframe.columns:
            dataframe[max_temperature_mean_column_name] = (
                dataframe.groupby(self.m_group_column, dropna=False)[self.m_max_temperature_column]
                  .rolling(self.m_max_temperature_window_days, min_periods=1)
                  .mean()
                  .reset_index(level=0, drop=True)
            )
        else:
            dataframe[max_temperature_mean_column_name] = np.nan

        return dataframe

    # Aplica transformações e cria as novas features
    def transform(self, dataframe, target=None):

        # Trabalha em DataFrame para manter nomes/índices
        dataframe = pd.DataFrame(dataframe).copy()

        # Estação do ano
        if self.m_date_column in dataframe.columns:
            # Garante dtype datetime
            dataframe[self.m_date_column] = pd.to_datetime(dataframe[self.m_date_column], errors='coerce')
            estacao = dataframe[self.m_date_column].dt.month.map(self.convert_month_to_season).astype('Int64')
            dataframe[season_column_name] = estacao.astype('float').astype('Int64')  # evita problemas de NaN -> imputar depois
            dataframe[season_column_name] = dataframe[season_column_name].astype('float')
        else:
            dataframe[season_column_name] = np.nan

        # Região
        if all(c in dataframe.columns for c in [self.m_latitude_column, self.m_longitude_column]):
            dataframe[region_column_name] = self.assign_dbscanlabels(dataframe).astype('int64')
        else:
            dataframe[region_column_name] = -1

        # Rollings temporais
        dataframe = self.add_temporal_rollings(dataframe)

        '''
        for col, dtype in dataframe.dtypes.items():
            print(f" - {col}: {dtype}")
        '''

        return dataframe


### 2. Tratamento de outliers

<u>**Transformação Logarítmica**</u>
- Aplica log(x) ou log(x+constante) para valores positivos
- Muito eficaz para dados com distribuição assimétrica positiva
- Comprime valores grandes e expande valores pequenos
- Fórmula: X_log = log(X + c), onde c evita log(0)

<u>**Transformação Raiz Quadrada**</u>
- Menos drástica que a transformação logarítmica
- Útil para dados de contagem e variáveis positivamente assimétricas
- Fórmula: X_sqrt = sqrt(X)

<u>**Winsorização (Capping/Clipping)**</u>

A **Winsorização** é uma técnica de tratamento de outliers que **limita valores extremos** sem removê-los completamente. Em vez de excluir outliers, substituímos os valores extremos pelos valores de percentis específicos.

**Como funciona:**
- Define-se limites baseados em percentis (ex: 5º e 95º percentil)
- Valores abaixo do limite inferior são substituídos pelo valor do limite inferior
- Valores acima do limite superior são substituídos pelo valor do limite superior

| Variável                                 | Melhor método          | Resultado obtido |
| :--------------------------------------- | :--------------------  | :-------------------------------------------------------------------------------------------------------- |
| **precipitacao**                         | **Log(x + 1)**         | Assimetria (7.87 → 2.59)                                                                                  |
| **umidade_relativa_max**                 | **Winsorização**       | Outliers (23 → 0)                                                                                         |
| **umidade_relativa_min**                 | **Winsorização**       | Outliers (1155 → 0)                                                                                       |
| **umidade_especifica**                   | **Sqrt**               | Simetria (0.89 → 0.16) / Outliers (6967 → 2102)                                                           |
| **radiacao_solar**                       | **Sem transformação**  |                                                                                                           |
| **temperatura_min**                      | **Winsorização**       | Outliers (5066 → 0)                                                                                       |
| **temperatura_max**                      | **Winsorização**       | Outliers (5066 → 0)                                                                                       |
| **velocidade_vento**                     | **Log(x + 1)**         | Assimetria (1.23 → 0.19) / Outliers (8723 → 1128)                                                         |
| **indice_queima**                        | **Winsorização**       |                                                                                                           |
| **umidade_combustivel_morto_100_horas**  | **Sqrt**               | Outliers (44 → 9)                                                                                         |
| **umidade_combustivel_morto_1000_horas** | **Sqrt**               | Outliers (373 → 4)                                                                                        |
| **componente_energia_lancada**           | **Sem transformação**  |                                                                                                           |
| **evapotranspiracao_real**               | **Sqrt**               | Assimetria (0.71 → -0.00) / Outliers (3292 → 153)                                                         |
| **evapotranspiracao_potencial**          | **Log(x + 1)**         | Outliers (679 → 0)                                                                                        |
| **deficit_pressao_vapor**                | **Log(x + 1)**         | Outliers (14758 → 672)                                                                                    |


In [6]:
# @title Classe OutliersTreatment

class OutliersTreatment(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.m_log_columns = [
            "precipitacao",
            "velocidade_vento",
            "evapotranspiracao_potencial",
            "deficit_pressao_vapor",
        ]
        self.m_sqrt_columns = [
            "umidade_especifica",
            "umidade_combustivel_morto_100_horas",
            "umidade_combustivel_morto_1000_horas",
            "evapotranspiracao_real",
        ]
        self.m_winsor_columns = [
            "umidade_relativa_max",
            "umidade_relativa_min",
            "temperatura_min",
            "temperatura_max",
            "indice_queima",
        ]
        self.m_winsor_limits = (0.05, 0.05)

    # Calcula parâmetros necessários para aplicar as transformações corretamente
    def fit(self, dataframe, target=None):

        # Garante que o usuário esteja enviado um dataframe no formato correto
        dataframe = dataframe if isinstance(dataframe, pd.DataFrame) else pd.DataFrame(dataframe)

        # Cálculo de offsets para garantir que não haverão valores zerados ou negativos
        # "coerce" convete valores inválidos para NaN

        self.m_log_offset = {}
        for column in self.m_log_columns:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                min_value = series.min()
                self.m_log_offset[column] = (abs(min_value) + 1) if pd.notna(min_value) and min_value <= 0 else 1.0

        self.m_sqrt_offset = {}
        for column in self.m_sqrt_columns:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                min_value = series.min()
                self.m_sqrt_offset[column] = (abs(min_value) + 0.01) if pd.notna(min_value) and min_value < 0 else 0.0

        # Garante que a winsorização só seja feita com colunas que realmente estão no dataframe
        actual_columns_to_winsor = [column for column in self.m_winsor_columns if column in dataframe.columns]
        low_quantile, high_quantile = self.m_winsor_limits

        if actual_columns_to_winsor:
            # Converte cada coluna para numérico (coerces -> NaN) e calcula quantis por coluna
            winsor_dataframe = dataframe[actual_columns_to_winsor].apply(pd.to_numeric, errors="coerce")
            self.m_low_quantile  = winsor_dataframe.quantile(low_quantile)
            self.m_high_quantile = winsor_dataframe.quantile(1 - high_quantile)
        else:
            # garante atributos vazios para não quebrar no transform()
            self.m_low_quantile  = pd.Series(dtype=float)
            self.m_high_quantile = pd.Series(dtype=float)

        return self

    # Aplica as transformações
    def transform(self, dataframe):

        # Garante que o usuário esteja enviado o dataframe correto
        dataframe = dataframe.copy() if isinstance(dataframe, pd.DataFrame) else pd.DataFrame(dataframe).copy()

        # LOG
        for column, offset in self.m_log_offset.items():
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = np.log(series + offset)

        # SQRT
        for column, offset in self.m_sqrt_offset.items():
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = np.sqrt(series + offset)

        # Winsorização
        for column in self.m_low_quantile.index:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = series.clip(lower=self.m_low_quantile[column], upper=self.m_high_quantile[column])

        return dataframe

### 3. Normalização dos dados

São utilizados para normalizar os dados numéricos
- SimpleImputer -> Preenche NaN com a média da feature
- MinMaxScaler -> Ajusta todos os valores para o intervalo entre 0 e 1

No momento, não estão sendo utilizadas features categóricas para treinar o modelo. Contudo, existe a implementação de uma normalização para as mesmas
- SimpleImputer -> Preenche NaN com a média da feature
- OneHotEncoder -> Transforma variáveis categóricas em binárias

In [7]:
# @title Classe DataNormalization

'''
categoric_cols = [season_column_name, region_column_name]

def numeric_columns_selector(dataframe):
    # Seleciona colunas numéricas, exceto as categóricas codificadas numericamente
    num = dataframe.select_dtypes(include='number').columns.tolist()
    return [column for column in num if column not in categoric_cols]

categoric_columns_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
])
'''

numeric_columns_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler()),
    ])

DataNormalization = ColumnTransformer(
    transformers=[
        ('numerical', numeric_columns_pipeline, make_column_selector(dtype_include='number')),
        #('categoric', categoric_columns_pipeline, categoric_cols)
    ],
    remainder='passthrough'  # mantém quaisquer colunas não listadas
)

### 4. Pipeline de pré-processamento

Concentra 5 passos
1. Criação de features
2. Tratamento de outliers
3. Eliminação de features indesejadas no treinamento dos modelos
4. Normalização dos dados
5. Sanitização dos dados


Para que o modelo seja treinado de maneira correta, além da coluna target, é ideal que algumas outras fiquem de fora de seu escopo, são elas:
- Data
- ID
- Latitude
- Longitude
- Estação do ano
- Região

In [8]:
# @title Classe ColumnDropper

# Elimina colunas não desejáveis no treinamento do modelo
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = [data_column_name, id_column_name, latitude_column_name, longitude_column_name, season_column_name, region_column_name, indice_queima]
    def fit(self, dataframe, target=None):
        return self
    def transform(self, dataframe):
        return dataframe.drop(columns=self.columns, errors="ignore")

Função simples que serve para garantir que os dados estão no formato esperado para que ocorra o treinamento dos modelos (numpy.ndarray)

In [9]:
# @title Função sanitizer

# Remove data e fire_id, além de converter o dataframe para o formato esperado pelos classificadores
def sanitizer(features):
    # Transforma em numpy.ndarray
    if isinstance(features, pd.DataFrame):
        features = features.select_dtypes(include='number')
    return features

In [10]:
# @title Pipeline preprocess

preprocess = Pipeline(steps=[
    ("features_creation", FeaturesCreation()),
    ("outliers_treatment", OutliersTreatment()),
    ("drop_unused_columns", ColumnDropper()),
    ("data_normalization", DataNormalization),
    ("sanitize", FunctionTransformer(sanitizer, validate=False, feature_names_out='one-to-one'))
])

### 5. Separação em treino/teste

<u>**Time Series Cross Validation**</u>

A Time Series Cross Validation é uma técnica especializada para validar modelos quando os dados possuem ordem cronológica. Diferente das técnicas tradicionais, ela respeita a estrutura temporal dos dados.

Foi implemetada uma função personalizada para que dados referentes ao mesmo incêndio permanecessem nos mesmos grupos

In [11]:
# @title Função group_time_series_cross_validation

# Gera folds (train_idx, test_idx) respeitando ordem temporal por grupo, embargo em nível de grupo e exclusão mútua treino/teste por grupo.
def group_time_series_cross_validation():
    dataframe = wildfires
    time_column = data_column_name
    group_column = id_column_name
    folds_amount = 5
    fold_groups_size = 1
    gap_between_groups_amount = 0

    # Ordena grupos pelo primeiro timestamp (data da primeira amostra do grupo)
    first_time = (
        dataframe[[group_column, time_column]]
        .dropna(subset=[time_column])
        .groupby(group_column)[time_column]
        .min()
        .sort_values()
    )
    ordered_groups = first_time.index.to_numpy()
    ordered_groups_len = len(ordered_groups)

    groups_amount_by_step = fold_groups_size

    min_train_groups = max(1, fold_groups_size) # pelo menos 1

    # Âncora: último grupo incluso no treino
    # Precisamos garantir espaço para gap + teste à frente
    max_anchor = ordered_groups_len - gap_between_groups_amount - fold_groups_size
    if max_anchor <= min_train_groups:
        return  # não há splits possíveis

    splits = 0
    anchor = min_train_groups
    while anchor <= max_anchor and splits < folds_amount:
        train_groups = ordered_groups[:anchor]

        test_start = anchor + gap_between_groups_amount
        test_end = test_start + fold_groups_size
        test_groups = ordered_groups[test_start:test_end]

        train_idx = dataframe.index[dataframe[group_column].isin(train_groups)].to_numpy()
        test_idx  = dataframe.index[dataframe[group_column].isin(test_groups)].to_numpy()

        if train_idx.size and test_idx.size:
            yield (train_idx, test_idx) # Geração sobre demanda (lazy evaluation)
            splits += 1

        anchor += groups_amount_by_step

cross_validation = cross_validation_splits = list(group_time_series_cross_validation())


### 6. Balanceamento dos dados

Após rodar um algoritmo que contava a quantidade de amostras de cada classe e também calculava a Razão de Desbalanceamento (IR), obtivemos:
- Classe 0 (não-incêndio): **2025** amostras acumuladas entre todas as folds (90%)
- Classe 1 (incêndio): **225** amostras acumuladas entre todas as folds (10%)
- Razão de desbalanceamento (IR) acumulada: **9.00x**

**Observação**: o dataset possui mais de 300.000 linhas, mas apenas 2.500 grupos distintos (separados por incêndio)

O dataset apresenta uma razão de desbalanceamento leve (IR < 10), o que não levanta a necessidade de utilização de algoritmos robustos para balanceamento.

Com essa informação em mente, foram feitos testes em alguns modelos que aceitavam como parâmetro o "peso" dos dados (class_weight = 'balanced') e os resultados foram esses:

**LogisticRegression** ✅
| Métrica   | Antes | Depois    | Diferença |
| --------- | ----- | --------- | --------- |
| Accuracy  | 0.921 | **0.932** | 🔼 +0.011 |
| Precision | 0.529 | **0.765** | 🔼 +0.236 |
| Recall    | 0.293 | **0.787** | 🔼 +0.494 |
| F1-score  | 0.335 | **0.715** | 🔼 +0.380 |
| ROC AUC   | 0.975 | **0.979** | 🔼 +0.004 |

**DecisionTreeClassifier** ✅
| Métrica   | Antes | Depois    | Diferença |
| --------- | ----- | --------- | --------- |
| Accuracy  | 0.909 | **0.940** | 🔼 +0.031 |
| Precision | 0.506 | **0.826** | 🔼 +0.320 |
| Recall    | 0.413 | **0.680** | 🔼 +0.267 |
| F1-score  | 0.391 | **0.662** | 🔼 +0.271 |
| ROC AUC   | 0.727 | **0.824** | 🔼 +0.097 |


**RandomForestClassifier** ✅
| Métrica   | Antes | Depois    | Diferença       |
| --------- | ----- | --------- | --------------- |
| Accuracy  | 0.925 | **0.933** | 🔼 +0.008       |
| Precision | 0.859 | **0.876** | 🔼 +0.017       |
| Recall    | 0.507 | **0.560** | 🔼 +0.053       |
| F1-score  | 0.498 | **0.566** | 🔼 +0.068       |
| ROC AUC   | 0.947 | 0.947     | ⚪ sem variação |

**XGBoost (scale_pos_weight=9.0)** 🟨
| Métrica   | Antes     | Depois    | Diferença |
| --------- | --------- | --------- | --------- |
| Accuracy  | **0.935** | 0.927     | 🔻 -0.008 |
| Precision | **0.830** | 0.797     | 🔻 -0.033 |
| Recall    | 0.653     | **0.693** | 🔼 +0.040 |
| F1-score  | 0.635     | **0.638** | 🔼 +0.003 |
| ROC AUC   | **0.950** | 0.946     | 🔻 -0.004 |


In [12]:
# @title Classe BalancingCount

# Classe que balanceia os dados
class BalancingCount(BaseEstimator, TransformerMixin):

    total_counts = None
    total_calls = 0

    def fit(self, dataframe, target=None):
        if target is None:
            return

        # Conta as ocorrências por classe neste fold
        counts = pd.Series(target).value_counts().sort_index()

        # Atualiza contagem total global
        if BalancingCount.total_counts is None:
            BalancingCount.total_counts = counts.copy()
        else:
            # Soma os valores por classe
            BalancingCount.total_counts = BalancingCount.total_counts.add(counts, fill_value=0)

        BalancingCount.total_calls += 1

        print(f"\nFold {BalancingCount.total_calls}")
        print("Contagem de classes (neste fold):")
        for classes, count in counts.items():
            print(f" Classe {classes}: {count} amostras")

        ratio = counts.max() / counts.min() if len(counts) > 1 else 1.0
        print(f"Razão de desbalanceamento (neste fold): {ratio:.2f}x")

        # --- Exibição acumulada ---
        if BalancingCount.total_calls == 1:
            print("\nContagem de classes em todos os folds")
        else:
            total_ratio = BalancingCount.total_counts.max() / BalancingCount.total_counts.min()
            print("\nContagem acumulada até então:")
            for classes, total in BalancingCount.total_counts.items():
                print(f"  Classe {int(classes)}: {int(total)} amostras acumuladas")
            print(f" Razão de desbalanceamento acumulada: {total_ratio:.2f}x")

        return self

    def transform(self, dataframe):
        return dataframe

    @classmethod
    def reset(cls):
        cls.total_counts = None
        cls.total_calls = 0

### 7. Treinamento dos modelos

Os modelos treinados foram
- Dummy (mais frequente)
- Regressão Logística
- Árvore de Decisão
- Random Forest
- Naive Bayes
- KNN
- Gradient Boosting
- AdaBoost
- HistGradientBoosting
- RidgeClassifier
- XGBoost
- CatBoost

<u>**Métricas para avaliação dos modelos**</u>

| **Métrica**                | **Descrição**                                                                               | **Interpretação ideal**                                                                  |
| -------------------------- | ------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------- |
| **Accuracy**               | Proporção de acertos totais (tanto positivos quanto negativos).                             | Boa para classes balanceadas, mas pode mascarar desempenho ruim em classes minoritárias. |
| **Precision**              | Entre as previsões positivas, quantas realmente eram positivas.                             | Alta precisão = poucos falsos positivos.                                                 |
| **Recall (Sensibilidade)** | Entre os casos realmente positivos, quantos foram identificados corretamente.               | Alto recall = poucos falsos negativos.                                                   |
| **F1-score**               | Média harmônica entre precisão e recall, equilibrando ambos.                                | Ideal quando há desbalanceamento de classes.                                             |
| **ROC AUC**                | Mede a capacidade global do modelo em separar as classes (0.5 = aleatório; 1.0 = perfeito). | Próximo de 1 indica boa separabilidade.                                                  |


In [13]:
# @title Escolha dos modelos e das métricas

modelos = {
    "Dummy (mais frequente)": DummyClassifier(strategy="most_frequent"), # Baseline simples

    # Sem balanceamento
    # "Regressão Logística": LogisticRegression(C=0.5, penalty='l2', solver='liblinear', max_iter=2000), # Regularização L2 leve
    # "Árvore de Decisão": DecisionTreeClassifier(max_depth=8, min_samples_split=4, min_samples_leaf=2, random_state=42), # Controle de profundidade e tamanho da folha
    # "Random Forest": RandomForestClassifier(n_estimators=150, max_depth=10, min_samples_split=5,random_state=42, n_jobs=-1), # Mais árvores e profundidade moderada

    # Com balanceamento
    "Regressão Logística": LogisticRegression(C=0.5, penalty='l2', solver='liblinear', max_iter=2000, class_weight = 'balanced'), # Regularização L2 leve
    "Árvore de Decisão": DecisionTreeClassifier(max_depth=8, min_samples_split=4, min_samples_leaf=2, random_state=42, class_weight = 'balanced'), # Controle de profundidade e tamanho da folha
    "Random Forest": RandomForestClassifier(n_estimators=150, max_depth=10, min_samples_split=5,random_state=42, n_jobs=-1, class_weight = 'balanced'), # Mais árvores e profundidade moderada

    "Naive Bayes": GaussianNB(var_smoothing=1e-8), # Suavização leve (mais estável)
    "KNN": KNeighborsClassifier(n_neighbors=7, weights='distance'), # Mais vizinhos e distância ponderada
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42), # Parâmetros leves conforme aula
    "AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42), # Taxa de aprendizado menor
    "HistGradientBoosting": HistGradientBoostingClassifier(max_iter=150, learning_rate=0.05, max_depth=5, random_state=42), # Mais iterações e taxa menor
    "RidgeClassifier": RidgeClassifier(alpha=1.0), # Regularização leve
    "XGBoost": XGBClassifier(eval_metric='logloss', n_estimators=150, learning_rate=0.05, max_depth=5, subsample=0.8, colsample_bytree=0.8, random_state=42), #  Parâmetros típicos de equilíbrio (aula 15 parte 4)
    "CatBoost": CatBoostClassifier(iterations=150, learning_rate=0.05, depth=5,verbose=0, random_state=42), # Taxa de aprendizado reduzida e iterações extras
}

# Métricas de avaliação
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0),
    'roc_auc': 'roc_auc'
}

In [14]:
# @title Algoritmo de treinamento

features = wildfires.drop(columns=target_column_name)
target = wildfires[target_column_name].astype(int)

periodos_testes = [30, 60, 90, 120, 150, 180, 210]

resultados_gerais = []

for periodo in periodos_testes:

    resultados = []

    window_days = periodo
    print("Período de dias: ", periodo)

    for nome, modelo in modelos.items():
        # print(f"Treinando modelo: {nome}...")
        # BalancingCount.reset()

        try:
            pipeline = Pipeline([
                ("preprocess", preprocess),
                ("nan_shield", SimpleImputer(strategy="constant", fill_value=0.0)), # Blindagem contra NaN
                # ("balancing", BalancingCount()),
                ("classificator", modelo)
            ])

            scores = cross_validate(
                pipeline,
                features,
                target,
                cv=cross_validation,
                scoring=scoring,
                n_jobs=1
            )

            resultados.append({
                "Modelo": nome,
                "Accuracy": np.mean(scores['test_accuracy']),
                "Precision": np.mean(scores['test_precision']),
                "Recall": np.mean(scores['test_recall']),
                "F1-score": np.mean(scores['test_f1']),
                "ROC AUC": np.mean(scores['test_roc_auc']),
            })

            resultados_gerais.append({
                "Modelo": nome,
                "F1-score": np.mean(scores['test_f1']),
                "Tempo": periodo
            })

            # print(f" Modelo {nome} treinado com sucesso.\n")

        except Exception as e:
            pass
            # print(f" Erro ao rodar o modelo {nome}: {e}\n")

    # Mostra os resultados por período
    df_resultados = pd.DataFrame(resultados).sort_values("F1-score", ascending=False)
    display(df_resultados)


Período de dias:  30


,Modelo,Accuracy,Precision,Recall,F1-score,ROC AUC
1,Regressão Logística,0.929333,0.761782,0.773333,0.696126,0.973432
4,Naive Bayes,0.905333,0.716920,0.653333,0.578059,0.952593
10,XGBoost,0.924000,0.869048,0.520000,0.549630,0.939062
3,Random Forest,0.928000,0.893750,0.506667,0.537889,0.947506
2,Árvore de Decisão,0.914667,0.770635,0.480000,0.535132,0.723358
5,KNN,0.928000,0.900000,0.480000,0.527165,0.873728
11,CatBoost,0.922667,0.888235,0.480000,0.497632,0.947062
8,HistGradientBoosting,0.922667,0.671569,0.493333,0.491209,0.963457
6,Gradient Boosting,0.901333,0.695652,0.426667,0.439888,0.797531
9,RidgeClassifier,0.916000,0.496774,0.373333,0.357101,0.988444


Período de dias:  60


,Modelo,Accuracy,Precision,Recall,F1-score,ROC AUC
1,Regressão Logística,0.925333,0.758333,0.746667,0.681081,0.974222
4,Naive Bayes,0.908000,0.722832,0.666667,0.586061,0.949333
5,KNN,0.926667,0.896552,0.466667,0.519975,0.874469
10,XGBoost,0.920000,0.885714,0.466667,0.489231,0.938667
8,HistGradientBoosting,0.921333,0.659664,0.493333,0.486462,0.957975
3,Random Forest,0.916000,0.857466,0.440000,0.440376,0.944099
11,CatBoost,0.917333,0.685714,0.440000,0.436290,0.945975
2,Árvore de Decisão,0.913333,0.664286,0.386667,0.428165,0.679259
7,AdaBoost,0.890667,0.762637,0.493333,0.391895,0.895802
9,RidgeClassifier,0.920000,0.503448,0.386667,0.380574,0.989827


Período de dias:  90


...

,Modelo,Accuracy,Precision,Recall,F1-score,ROC AUC
1,Regressão Logística,0.933333,0.770000,0.760000,0.695119,0.973037
10,XGBoost,0.933333,0.885714,0.600000,0.617886,0.937086
8,HistGradientBoosting,0.934667,0.652941,0.640000,0.605810,0.971309
7,AdaBoost,0.921333,0.822078,0.573333,0.543507,0.895802
5,KNN,0.922667,0.892857,0.426667,0.488787,0.843852
4,Naive Bayes,0.902667,0.520854,0.586667,0.488443,0.949185
6,Gradient Boosting,0.916000,0.841231,0.400000,0.462623,0.767309
2,Árvore de Decisão,0.921333,0.678261,0.400000,0.457907,0.689630
3,Random Forest,0.918667,0.888235,0.440000,0.454508,0.933580
11,CatBoost,0.908000,0.625714,0.426667,0.352059,0.945086


Período de dias:  180


,Modelo,Accuracy,Precision,Recall,F1-score,ROC AUC
1,Regressão Logística,0.933333,0.770000,0.760000,0.695119,0.973037
10,XGBoost,0.933333,0.885714,0.600000,0.617886,0.937086
8,HistGradientBoosting,0.934667,0.652941,0.640000,0.605810,0.971309
7,AdaBoost,0.921333,0.822078,0.573333,0.543507,0.895802
5,KNN,0.922667,0.892857,0.426667,0.488787,0.843852
4,Naive Bayes,0.902667,0.520854,0.586667,0.488443,0.949185
6,Gradient Boosting,0.916000,0.841231,0.400000,0.462623,0.767309
2,Árvore de Decisão,0.921333,0.678261,0.400000,0.457907,0.689630
3,Random Forest,0.918667,0.888235,0.440000,0.454508,0.933580
11,CatBoost,0.908000,0.625714,0.426667,0.352059,0.945086


Período de dias:  210


,Modelo,Accuracy,Precision,Recall,F1-score,ROC AUC
1,Regressão Logística,0.933333,0.770000,0.760000,0.695119,0.973037
10,XGBoost,0.933333,0.885714,0.600000,0.617886,0.937086
8,HistGradientBoosting,0.934667,0.652941,0.640000,0.605810,0.971309
7,AdaBoost,0.921333,0.822078,0.573333,0.543507,0.895802
5,KNN,0.922667,0.892857,0.426667,0.488787,0.843852
4,Naive Bayes,0.902667,0.520854,0.586667,0.488443,0.949185
6,Gradient Boosting,0.916000,0.841231,0.400000,0.462623,0.767309
2,Árvore de Decisão,0.921333,0.678261,0.400000,0.457907,0.689630
3,Random Forest,0.918667,0.888235,0.440000,0.454508,0.933580
11,CatBoost,0.908000,0.625714,0.426667,0.352059,0.945086


In [15]:
# @title Comparação de resultados

resultados_por_modelo = {}

# Agrupa os resultados por modelo
for entrada in resultados_gerais:
    nome_modelo = entrada["Modelo"]
    if nome_modelo not in resultados_por_modelo:
        resultados_por_modelo[nome_modelo] = []
    resultados_por_modelo[nome_modelo].append(entrada)

# Ordena os resultados de cada modelo
for nome_modelo, lista_resultados in resultados_por_modelo.items():
    lista_ordenada = sorted(
        lista_resultados,
        key=lambda x: x["F1-score"],
        reverse=True
    )

    print(f"\n==== Modelo: {nome_modelo} ====")
    for posicao, item in enumerate(lista_ordenada, start=1):
        print(f"{posicao}º lugar | Tempo: {item['Tempo']:>3} dias | F1-score: {item['F1-score']:.4f}")

# Ordena os resultados de maneira gerais
resultados_gerais_ordenados = sorted(
    resultados_gerais,
    key=lambda x: x["F1-score"],
    reverse=True
)

print("\n==== Ranking geral (todos os modelos e períodos) ====")
for posicao, item in enumerate(resultados_gerais_ordenados, start=1):
    print(f"{posicao}º lugar | Modelo: {item['Modelo']} | Tempo: {item['Tempo']:>3} dias | F1-score: {item['F1-score']:.4f}")


==== Modelo: Dummy (mais frequente) ====
1º lugar | Tempo:  30 dias | F1-score: 0.0000
2º lugar | Tempo:  60 dias | F1-score: 0.0000
3º lugar | Tempo:  90 dias | F1-score: 0.0000
4º lugar | Tempo: 120 dias | F1-score: 0.0000
5º lugar | Tempo: 150 dias | F1-score: 0.0000
6º lugar | Tempo: 180 dias | F1-score: 0.0000
7º lugar | Tempo: 210 dias | F1-score: 0.0000

==== Modelo: Regressão Logística ====
1º lugar | Tempo:  90 dias | F1-score: 0.6988
2º lugar | Tempo: 120 dias | F1-score: 0.6979
3º lugar | Tempo:  30 dias | F1-score: 0.6961
4º lugar | Tempo: 150 dias | F1-score: 0.6951
5º lugar | Tempo: 180 dias | F1-score: 0.6951
6º lugar | Tempo: 210 dias | F1-score: 0.6951
7º lugar | Tempo:  60 dias | F1-score: 0.6811

==== Modelo: Árvore de Decisão ====
1º lugar | Tempo:  90 dias | F1-score: 0.6613
2º lugar | Tempo:  30 dias | F1-score: 0.5351
3º lugar | Tempo: 120 dias | F1-score: 0.4804
4º lugar | Tempo: 150 dias | F1-score: 0.4579
5º lugar | Tempo: 180 dias | F1-score: 0.4579
6º lugar

O modelo escolhido foi a **Regressão Logística**.

Justificativas para a escolha:
- Ele apresentou o **melhor F1-score** entre todos os modelos avaliados, indicando melhor equilíbrio entre precisão e recall.
- Também teve **alta AUC-ROC**, mostrando excelente separação entre classes.
- Com o parâmetro `class_weight='balanced'`, lidou bem com o desbalanceamento da base de incêndios (poucos incêndios vs muitos "não incêndios").
- É um modelo **interpretável**, ideal para aplicar técnicas de explicabilidade
- Seus coeficientes permitem identificar claramente a influência das variáveis, atendendo à tarefa de "Aplicar algoritmos para ajudar a explicar as decisões do modelo".

<b>Explicabilidade do modelo</b>
A técnica usada aqui é a análise dos **coeficientes** da Regressão Logística.
Interpretação:
- Coeficientes POSITIVOS aumentam a probabilidade de incêndio.
- Coeficientes NEGATIVOS reduzem essa probabilidade.
- Quanto maior o valor absoluto do coeficiente, maior a importância da variável.

In [16]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Define e treina o pipeline
pipeline = Pipeline(steps=[
    ("preprocess", preprocess),  # já criado no projeto
    ("nan_shield", SimpleImputer(strategy="constant", fill_value=0.0)),
    ("classificator", LogisticRegression(
        C=0.5, penalty='l2', solver='liblinear',
        max_iter=2000, class_weight='balanced'
    ))
])

# Treina o modelo com os dados
pipeline.fit(features, target)

# Aplica o pré-processamento
X_preprocessado = pipeline.named_steps["preprocess"].transform(features)

# Identifica as colunas que permanecem após o pré-processamento
# Baseado na lógica de FeaturesCreation e ColumnDropper

# Colunas originais no DataFrame 'features'
original_feature_cols = features.columns.tolist()

# Colunas adicionadas por FeaturesCreation (que são numéricas)
added_numeric_cols_by_feature_creation = [
    precipitation_sum_window_column_name,
    max_temperature_mean_column_name
] # estacao_ano_id e regiao_incendio são criadas mas depois dropadas pelo ColumnDropper

# Colunas a serem removidas pelo ColumnDropper
dropped_cols_names = [
    data_column_name,
    id_column_name,
    latitude_column_name,
    longitude_column_name,
    season_column_name, # 'estacao_ano_id'
    region_column_name, # 'regiao_incendio'
    indice_queima
]

# Construir a lista de nomes de colunas que realmente estarão no X_preprocessado
# Começamos com as colunas numéricas originais que não são dropadas,
# e adicionamos as novas colunas numéricas que também não são dropadas.

# Colunas numéricas originais em 'features' (excluindo as que serão dropadas)
numeric_cols_from_original_not_dropped = [
    col for col in features.select_dtypes(include='number').columns
    if col not in dropped_cols_names
]

# As colunas finais serão a união das numéricas originais não dropadas
# e as novas numéricas criadas e não dropadas.
final_column_names = numeric_cols_from_original_not_dropped + added_numeric_cols_by_feature_creation

# Garante que não há duplicatas e mantém a ordem (melhora para o warning pd.unique)
final_column_names = list(dict.fromkeys(final_column_names))

# Verifica se o número de colunas corresponde
if len(final_column_names) != X_preprocessado.shape[1]:
    print(f"Aviso: O número de nomes de coluna calculados ({len(final_column_names)}) não corresponde ao número de colunas pré-processadas ({X_preprocessado.shape[1]}).")

X_preprocessado_df = pd.DataFrame(X_preprocessado, columns=final_column_names)

# Extrai os coeficientes da regressão logística
modelo = pipeline.named_steps["classificator"]
coeficientes = pd.Series(modelo.coef_[0], index=X_preprocessado_df.columns)

# Ordena pelos mais influentes (positiva ou negativamente)
coef_ordenados = coeficientes.sort_values(key=np.abs, ascending=False)

# Exibe as 10 variáveis mais influentes
print(" Top 10 variáveis que mais influenciam a decisão do modelo:")
print(coef_ordenados.head(10))

#Interpretação das 10 variáveis mais influentes segundo a Regressão Logística

# Isso indica as que mais influenciam a decisão do modelo (positiva ou negativamente).
# - Coeficientes POSITIVOS: aumentam a chance de incêndio.
# - Coeficientes NEGATIVOS: reduzem a chance de incêndio.

# Explicação de cada uma:

# 1. soma_precipitacao_90_dias (+13.28):
#    Mais chuva acumulada pode favorecer crescimento de vegetação, que seca e se torna combustível, aumentando o risco.

# 2. umidade_combustivel_morto_1000_horas (−10.17):
#    Alta umidade em materiais mortos profundos (ex: troncos) dificulta propagação do fogo.

# 3. evapotranspiracao_real (+3.22):
#    Indica perda real de água do solo/vegetação. Quanto maior, mais seco o ambiente, maior risco.

# 4. evapotranspiracao_potencial (−2.70):
#    Pode representar ambientes naturalmente úmidos, onde o fogo é menos provável.

# 5. componente_energia_lancada (−2.63):
#    Energia irradiada da superfície. Pode estar relacionada a estabilidade térmica, dificultando ignição.

# 6. umidade_combustivel_morto_100_horas (+2.20):
#    Baixa umidade em galhos/folhas secas favorece ignição rápida.

# 7. media_temp_max_90_dias (−1.79):
#    Pode indicar sazonalidade: incêndios ocorrem mais em épocas frias/secas que em calor úmido (região-dependente).

# 8. deficit_pressao_vapor (−1.33):
#    Medida da secura do ar. Apesar de esperarmos um coeficiente positivo, o modelo pode ter capturado um padrão contrário.

# 9. umidade_relativa_min (−1.01):
#    Umidade mínima do ar no dia. Quanto menor, mais seco — mas pode estar correlacionado a outras variáveis que reduzem o risco.

#10. precipitacao (−0.97):
#    Chuva recente reduz o risco de incêndio no mesmo dia.

 Top 10 variáveis que mais influenciam a decisão do modelo:
soma_precipitacao_90_dias               13.278625
umidade_combustivel_morto_1000_horas   -10.169172
evapotranspiracao_real                   3.224891
evapotranspiracao_potencial             -2.702668
componente_energia_lancada              -2.631507
umidade_combustivel_morto_100_horas      2.197365
media_temp_max_90_dias                  -1.794647
deficit_pressao_vapor                   -1.329952
umidade_relativa_min                    -1.010543
precipitacao                            -0.971644
dtype: float64
